In [2]:
import os
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
import json

# 1. データベースの準備 (SQLite)
dbname = 'google_repos.db'
conn = sqlite3.connect(dbname)
cur = conn.cursor()

# テーブルが存在しなければ作成 (id, name, language, stars)
# nameにUNIQUE制約をつけて重複挿入を避ける
cur.execute('''
    CREATE TABLE IF NOT EXISTS repositories (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT UNIQUE,
        language TEXT,
        stars INTEGER
    )
''')
conn.commit()

# 2. スクレイピングの実行 (GitHub API)
url = "https://api.github.com/orgs/google/repos"
params = {
    'per_page': 100,  # 1ページあたりの取得件数
    'page': 1
}

# ヘッダー: 明示的にJSONを要求。必要なら環境変数GITHUB_TOKENで認証トークンをセット可能
headers = {
    'User-Agent': 'Mozilla/5.0',
    'Accept': 'application/vnd.github.v3+json'
}
token = os.environ.get('GITHUB_TOKEN')
if token:
    headers['Authorization'] = f'token {token}'

print("GitHubのGoogleリポジトリ情報を取得中...")

def safe_parse_json(text):
    """
    応答がプレフィックスを含んでいたり空文字の場合に備えて、
    テキストから最初の JSON オブジェクト/配列を見つけて解析を試みる。
    成功すればPythonオブジェクトを返す。失敗すればNoneを返す。
    """
    if not text or not text.strip():
        return None
    txt = text.lstrip('\ufeff')  # BOM除去
    # XSSI等のプレフィックス対処: 最初の '{' または '[' を探す
    idx_obj = min([i for i in (txt.find('{'), txt.find('[')) if i != -1], default=-1)
    if idx_obj > 0:
        txt = txt[idx_obj:]
    try:
        return json.loads(txt)
    except json.JSONDecodeError:
        return None

try:
    page = 1
    total_fetched = 0
    while True:
        params['page'] = page
        try:
            response = requests.get(url, params=params, headers=headers, timeout=10)
        except requests.RequestException as e:
            print(f"リクエストエラー (page {page}): {e}")
            break

        if response.status_code == 200:
            content_type = response.headers.get('Content-Type', '')
            repos_page = None

            # 空レスポンスやJSONでない場合に備えて安全にパース
            if not response.text or not response.text.strip():
                repos_page = []
                print(f"page {page}: 空のレスポンスを受信しました。終了します。")
            elif 'application/json' in content_type:
                try:
                    repos_page = response.json()
                except json.JSONDecodeError:
                    # まれにプレフィックス等でjson()が失敗することがあるためフォールバック
                    repos_page = safe_parse_json(response.text)
                    if repos_page is None:
                        print(f"page {page}: JSONデコードに失敗しました。レスポンス先頭: {response.text[:500]}")
                        repos_page = []
            else:
                # Content-TypeがJSONでない場合にも試行的に解析（GitHubがHTMLを返す時はエラーページ）
                repos_page = safe_parse_json(response.text)
                if repos_page is None:
                    print(f"page {page}: 期待するJSONが返ってきませんでした。Content-Type: {content_type}")
                    print("レスポンス先頭:", response.text[:1000])
                    repos_page = []

            if not isinstance(repos_page, list):
                # API仕様が変わって辞書を返す場合などを安全に扱う
                print(f"page {page}: 期待した配列ではありません。型: {type(repos_page)}")
                repos_page = []

            if not repos_page:
                print(f"page {page}: 取得したリポジトリは0件です。終了します。")
                break

            print(f"page {page}: {len(repos_page)} 件のリポジトリを処理します...")
            for repo in repos_page:
                try:
                    repo_name = repo.get('name', 'Unknown')
                    language = repo.get('language') or 'No Language'
                    stars = repo.get('stargazers_count') or 0

                    print(f"取得: {repo_name} | {language} | {stars} stars")

                    # UNIQUE制約があるため重複を無視するINSERT OR IGNOREを使う
                    cur.execute(
                        'INSERT OR IGNORE INTO repositories (name, language, stars) VALUES (?, ?, ?)',
                        (repo_name, language, stars)
                    )
                    # もし既存レコードのスター数を最新に更新したければ以下を使う:
                    # cur.execute('INSERT INTO repositories (name, language, stars) VALUES (?, ?, ?) ON CONFLICT(name) DO UPDATE SET language=excluded.language, stars=excluded.stars', (repo_name, language, stars))

                    time.sleep(0.05)
                    total_fetched += 1
                except Exception as e:
                    print(f"レコード処理中のエラー (repo {repo.get('name')}): {e}")
                    continue

            conn.commit()
            page += 1
            # 安全のためページ上限を設ける（必要であれば調整）
            if page > 10:
                print("ページ上限に達しました。中断します。")
                break

        elif response.status_code == 403:
            rem = response.headers.get('X-RateLimit-Remaining')
            reset = response.headers.get('X-RateLimit-Reset')
            print("APIが403を返しました。レート制限の可能性があります。")
            print("X-RateLimit-Remaining:", rem, "X-RateLimit-Reset:", reset)
            print("レスポンス先頭:", response.text[:1000])
            break
        else:
            print(f"APIリクエスト失敗 (status {response.status_code}): レスポンス先頭: {response.text[:1000]}")
            break

    print(f"合計 {total_fetched} 件のリポジトリを取得してデータベースに格納しました。")
except Exception as e:
    print(f"予期しないエラー: {e}")

# 3. 保存したデータをSELECT文で表示する (課題の要件)
print("\n--- データベース保存結果 ---")
try:
    cur.execute('SELECT * FROM repositories ORDER BY stars DESC')
    rows = cur.fetchall()

    print(f"合計 {len(rows)} 件のリポジトリを保存しました。\n")

    for row in rows:
        print(f"ID: {row[0]}, リポジトリ名: {row[1]}, 言語: {row[2]}, スター数: {row[3]}")
finally:
    # 4. コネクションを閉じる
    conn.close()


GitHubのGoogleリポジトリ情報を取得中...
page 1: 100 件のリポジトリを処理します...
取得: truth | Java | 2778 stars
取得: ruby-openid-apps-discovery | Ruby | 38 stars
取得: autoparse | Ruby | 146 stars
取得: anvil-build | Python | 58 stars
page 1: 100 件のリポジトリを処理します...
取得: truth | Java | 2778 stars
取得: ruby-openid-apps-discovery | Ruby | 38 stars
取得: autoparse | Ruby | 146 stars
取得: anvil-build | Python | 58 stars
取得: googletv-android-samples | Java | 165 stars
取得: ChannelPlate | JavaScript | 23 stars
取得: GL-Shader-Validator | Python | 40 stars
取得: qpp | JavaScript | 43 stars
取得: googletv-android-samples | Java | 165 stars
取得: ChannelPlate | JavaScript | 23 stars
取得: GL-Shader-Validator | Python | 40 stars
取得: qpp | JavaScript | 43 stars
取得: CSP-Validator | Python | 27 stars
取得: embed-dart-vm | C++ | 99 stars
取得: module-server | JavaScript | 565 stars
取得: cxx-std-draft | TeX | 53 stars
取得: CSP-Validator | Python | 27 stars
取得: embed-dart-vm | C++ | 99 stars
取得: module-server | JavaScript | 565 stars
取得: cxx-std-draft | T